In [93]:
import multiprocessing
import re
from io import StringIO
import uuid

import pandas as pd
import numpy as np

from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

In [94]:
target_name = 'SEIZ_updated'
# n_runs = 5
# max_parallel_runs = 8
# threads_per_run = multiprocessing.cpu_count()//min(n_runs,max_parallel_runs)
threads_per_run = multiprocessing.cpu_count()
init_exposures = 100

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  ContinuousParameter('epsilon', 2, 2),
                                  DiscreteParameter('prop_init_skeptics', np.arange(1, init_exposures-1, 1, dtype=np.int_))])

In [95]:
def run_job(p, l, epsilon, init_skeptics, init_adopters, clear_results=False):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    vars = {
        '{p}': p, 
        '{l}': l, 
        '{epsilon}': epsilon, 
        '{init_skeptic}': init_skeptics, 
        '{init_adopt}': init_adopters
    }

    for target, value in vars.items():
        filedata = filedata.replace(target, str(value))

    with open(f'{target_name}_out.fred', 'w') as file:
        file.write(filedata)
        
    if clear_results:
        !yes | fred_clear_all_results
        
    run_uuid = uuid.uuid4()
    
    # !fred_job -p {target_name}_out.fred -k {target_name}_{p}_{l}_{epsilon}_run -n {n_runs} -m {max_parallel_runs} -t {threads_per_run}
    !fred_job -p {target_name}_out.fred -k {target_name}_{run_uuid}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{run_uuid}_run > results/{target_name}_{run_uuid}_run.csv

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'r') as file:
        lines = file.readlines()

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'w') as file:
        lines[0] = ', '.join(map(lambda item: f'{item[0][1:-1]}={item[1]}', vars.items()))+'\n'
        file.writelines(lines)

    # with open(f'results/{target_name}_{p}_{l}_{epsilon}_run.csv', 'r') as csv:
    #     results = csv.read()
    # 
    # results = re.split('Run,\d\n', results)[1:]
    # return [pd.read_csv(StringIO(data)) for data in results]
    return pd.read_csv(f'results/{target_name}_{run_uuid}_run.csv', header=2)

In [96]:
def sample(X, **kwargs):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != len(parameter_space._parameters):
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be {len(parameter_space._parameters)})')
    
    Y = []

    for (p, l, epsilon, init_skeptics) in X: # TODO: parallelize this?
        data = run_job(p, l, epsilon, init_skeptics, init_exposures-init_skeptics, **kwargs)
        # Y.append([run['ADOPT.I'].iat[-1] for run in data])
        Y.append([data['ADOPT.I'].iat[-1]])
        
    return np.vstack(Y)

In [97]:
design = RandomDesign(parameter_space)
num_data_points = 5
X = design.get_samples(num_data_points)

In [98]:
X

array([[ 0.13773894,  0.61219263,  2.        , 78.        ],
       [ 0.658957  ,  0.63596732,  2.        , 44.        ],
       [ 0.38752622,  0.49432402,  2.        ,  3.        ],
       [ 0.68030643,  0.62002139,  2.        ,  5.        ],
       [ 0.72109157,  0.9696176 ,  2.        , 66.        ]])

In [99]:
Y = sample(X, clear_results=True)

You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_44fbcfde-8a8d-4518-91bd-5ad38a05d2fa_run 1
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/1/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/1/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/1/OUT/RUN1/LOG)
You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wa

In [100]:
Y

array([[32391],
       [38665],
       [45002],
       [44555],
       [ 8595]])

In [101]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [102]:
expected_improvement = ExpectedImprovement(model=model_emukit)

In [103]:
bayesopt_loop = BayesianOptimizationLoop(model=model_emukit,
                                         space=parameter_space,
                                         acquisition=expected_improvement,
                                         batch_size=1)

In [104]:
max_iterations = 30
bayesopt_loop.run_loop(sample, max_iterations)

 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0173f224-733e-4cdc-8fc5-bf5cfab8da99_run 2
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/2/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/2/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/2/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_cbba391a-63dc-4c4c-8a9c-b5cfa8eb9a0c_run 3
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/3/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/3/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/3/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


...

 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_b869d948-160a-4307-9d4d-8f1e19b79b51_run 30
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/30/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/30/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/30/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_501c1f8c-b376-48e6-a5b0-df4fb4b4fb3f_run 31
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/31/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/31/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/31/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


In [105]:
results = bayesopt_loop.get_results()

In [106]:
print(results.minimum_location, results.minimum_value)
print(results.best_found_value_per_iteration)

[ 0.72109157  0.9696176   2.         66.        ] 8595.0
[32391. 32391. 32391. 32391.  8595.  8595.  8595.  8595.  8595.  8595.
  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.
  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.  8595.
  8595.  8595.  8595.  8595.  8595.]
